In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import json
import time
import sys
import numpy as np

attr_names = []
for name in open('dataset/attribute/attribute_data/attribute_names.txt').readlines() :
    attr_names.append(name.strip())

loss_weights = []
for line in open('dataset/attribute/attribute_data/balancing_weights.txt').readlines() :
    pl, nl = line.strip().split('\t')
    loss_weights.append( float(pl) / ( float(pl) + float(nl) ) )

while(True):
    try:
        log = open('snapshots/aPascal_weight_balance/training.log','r')


        x = []
        loss = []
        la = []
        
        ev_x = []
        acc = []
        pre = []
        recall = []
        f1 = []
        
        
        for i in range(0,64) :
            la.append([])
            acc.append([])
            pre.append([])
            recall.append([])
            f1.append([])

        for line in log.readlines() :
            c = json.loads(line)
            if (c['type'] == 'train') :
                x.append( c['contents']['state']['nEvalCounter'] )
                loss.append( c['contents']['total_loss'] )
                for i in range(0,64) :
                    la[i].append( c['contents']['loss_per_attributes'][i] * (loss_weights[i]))
            elif (c['type'] == 'eval') :
                ev_x.append( c['contents']['state']['nEvalCounter'] )
                
                tp = np.array(c['contents']['eval']['tp'],np.float)
                tn = np.array(c['contents']['eval']['tn'],np.float)
                fp = np.array(c['contents']['eval']['fp'],np.float)
                fn = np.array(c['contents']['eval']['fn'],np.float)
                total = (tp + tn + fp + fn)
                
                c_acc = (tp + tn) / total
                c_pre = tp / (tp + fp)
                c_recall = tp / (tp+ fn)
                c_f1 = 2 * c_pre * c_recall / (c_pre + c_recall)
                
                for i in range(0,64) :    
                    acc[i].append(c_acc[i])
                    pre[i].append(c_pre[i])
                    recall[i].append(c_recall[i])
                    f1[i].append(c_f1[i])

        plt.close("all")
        f,axes = plt.subplots(3,2)
        f.set_size_inches(20,20)
        axes[0,0].set_title('Total Loss')
        axes[0,1].set_title('Loss per attribute')
        axes[1,0].set_title('Accuracy per attribute')
        axes[1,1].set_title('Precision per attribute')
        axes[2,0].set_title('Recall per attribute')
        axes[2,1].set_title('F1 per attribute')
        
        axes[0,0].plot(x,loss)


        for i in range(0,64) :
            axes[0,1].plot(x, la[i] )
            axes[1,0].plot(ev_x,acc[i])
            axes[1,1].plot(ev_x,pre[i])
            axes[2,0].plot(ev_x,recall[i])
            axes[2,1].plot(ev_x,f1[i])

        display.clear_output(wait=True)
        display.display(plt.gcf())
        
        if( len(ev_x) != 0 ):
            
            
            res = []
            for i in range(0,64) :
                res.append( f1[i][-1] )
            
            kk = np.argsort( res )[::-1]
            
            print '%20s\t%4s\t%4s\t%4s\t%4s'%('attr','f1','acc','pre','recall')
            for k in kk :
                print '%20s\t%0.3f\t%0.3f\t%0.3f\t%0.3f'%(attr_names[k]+'('+str(k+1)+')', f1[k][-1], acc[k][-1], pre[k][-1], recall[k][-1])
            sys.stdout.flush()
        
    #     plt.show()    
        time.sleep(10.0)
    except KeyboardInterrupt:
        plt.close("all")
        break
    